In [79]:
import random
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import trange
from torch.autograd import Variable

In [80]:
import gensim.downloader
word2vec_google_news = gensim.downloader.load('word2vec-google-news-300')

In [81]:
def read_conll_file(file_path):
    sentences = []
    sentence = []
    tags = []
    tag = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if len(line) == 0:  # Empty line indicates the end of a sentence
                if sentence:
                    sentences.append(sentence)
                    tags.append(tag)
                sentence = []
                tag = []
            else:
                parts = line.split()
                word = parts[0]
                if word == '-DOCSTART-':
                    continue
                ner_tag = parts[-1]
                sentence.append(word)
                tag.append(ner_tag)
        if sentence:
            sentences.append(sentence)
            tags.append(tag)
    return sentences, tags

In [82]:
train_data, train_tags = read_conll_file('CoNLL2003_dataset/eng.train')
val_data, val_tags = read_conll_file('CoNLL2003_dataset/eng.testa')
test_data, test_tags = read_conll_file('CoNLL2003_dataset/eng.testb')

In [97]:
from itertools import chain

def get_unique_words(allSentences):
    uniqueWords = [] 
    flatten_sentences = list(chain(*allSentences))
    for i in flatten_sentences:
        if not i in uniqueWords:
            uniqueWords.append(i);
    uniqueWords.append('<pad>')
    uniqueWords.append('UNK')
    return uniqueWords

def get_unique_tags(allTags):
    uniqueWords = []
    flatten_sentences = list(chain(*allTags))
    for i in flatten_sentences:
        if not i in uniqueWords:
            uniqueWords.append(i);
    return uniqueWords

In [105]:
unique_words = get_unique_words(train_data+val_data) # test_data not included
unique_tags = get_unique_tags(train_tags+val_tags+test_tags)

In [106]:
tag_map = {}
for index, tag in enumerate(unique_tags):
    tag_map[tag] = index
tag_map

{'I-ORG': 0,
 'O': 1,
 'I-MISC': 2,
 'I-PER': 3,
 'I-LOC': 4,
 'B-LOC': 5,
 'B-MISC': 6,
 'B-ORG': 7}

In [111]:
word2vec_goog1e_news: gensim.models.keyedvectors.KeyedVectors = gensim.downloader.load('word2vec-google-news-300')
weights = torch.FloatTensor(word2vec_goog1e_news.vectors)
embedding = nn.Embedding.from_pretrained(weights)
WORD = 'UNK'
# WORD = '<pad>'
WORDID = word2vec_goog1e_news.key_to_index[WORD]
WORDID

98307

In [ ]:
# train_sentences_file = "stanford_cs230_medium_data/train/sentences.txt"
# train_labels_file = "stanford_cs230_medium_data/train/labels.txt"
train_sentences_file = "stanford_cs230_small_data/test/sentences.txt"
train_labels_file = "stanford_cs230_small_data/test/labels.txt"

train_sentences = []
train_labels = []

with open(train_sentences_file, encoding="utf-8") as f:
    for sentence in f.read().splitlines():
        #replace each token by its index if it is in vocab
        #else use index of UNK
        s = [vocab[token] if token in vocab
             else vocab['UNK']
             for token in sentence.split(' ')]
        train_sentences.append(s)

with open(train_labels_file) as f:
    for sentence in f.read().splitlines():
        #replace each label by its index
        l = [tag_map[label] for label in sentence.split(' ')]
        train_labels.append(l)
        
print(f"train_sentences: {train_sentences}")
print(f"train_labels: {train_labels}")

train_sentences: [[266, 9, 267, 1, 268, 269, 11, 270, 93, 271, 272, 273, 274, 275, 165, 276, 78, 277, 167, 9, 6, 278, 21], [279, 280, 281, 282, 283, 237, 284, 285, 21], [286, 287, 278, 63, 9, 288, 289, 137, 11, 290, 24, 291, 292, 13, 293, 294, 21], [295, 296, 297, 298, 245, 169, 63, 299, 300, 7, 301, 302, 63, 303, 7, 304, 305, 306, 13, 307, 296, 297, 308, 21], [61, 309, 310, 311, 312, 303, 165, 125, 18, 9, 313, 1, 314, 299, 315, 316, 115, 116, 317, 318, 262, 319, 11, 320, 31, 303, 21], [321, 165, 300, 317, 318, 322, 45, 323, 324, 63, 303, 167, 9, 325, 315, 326, 327, 21], [61, 298, 180, 328, 310, 300, 7, 329, 9, 330, 192, 331, 332, 167, 303, 68, 184, 333, 21], [61, 297, 298, 334, 303, 68, 335, 1, 336, 11, 337, 338, 339, 13, 296, 297, 308, 93, 340, 341, 342, 343, 344, 11, 303, 21], [345, 125, 126, 346, 3, 24, 45, 347, 1, 9, 348, 349, 350, 167, 351, 345, 352, 353, 354, 355, 21], [198, 126, 356, 357, 59, 358, 359, 133, 346, 360, 150, 361, 362, 9, 363, 364, 1, 365, 21]]
train_labels: [[0, 0

In [ ]:
# val_sentences_file = "stanford_cs230_medium_data/val/sentences.txt"
# val_labels_file = "stanford_cs230_medium_data/val/labels.txt"
val_sentences_file = "stanford_cs230_small_data/test/sentences.txt"
val_labels_file = "stanford_cs230_small_data/test/labels.txt"

val_sentences = []
val_labels = []

with open(val_sentences_file, encoding="utf-8") as f:
    for sentence in f.read().splitlines():
        #replace each token by its index if it is in vocab
        #else use index of UNK
        s = [vocab[token] if token in vocab
             else vocab['UNK']
             for token in sentence.split(' ')]
        val_sentences.append(s)

with open(val_labels_file, encoding="utf-8") as f:
    for sentence in f.read().splitlines():
        #replace each label by its index
        l = [tag_map[label] for label in sentence.split(' ')]
        val_labels.append(l)

print(f"val_sentences: {val_sentences}")
print(f"val_labels: {val_labels}")

val_sentences: [[266, 9, 267, 1, 268, 269, 11, 270, 93, 271, 272, 273, 274, 275, 165, 276, 78, 277, 167, 9, 6, 278, 21], [279, 280, 281, 282, 283, 237, 284, 285, 21], [286, 287, 278, 63, 9, 288, 289, 137, 11, 290, 24, 291, 292, 13, 293, 294, 21], [295, 296, 297, 298, 245, 169, 63, 299, 300, 7, 301, 302, 63, 303, 7, 304, 305, 306, 13, 307, 296, 297, 308, 21], [61, 309, 310, 311, 312, 303, 165, 125, 18, 9, 313, 1, 314, 299, 315, 316, 115, 116, 317, 318, 262, 319, 11, 320, 31, 303, 21], [321, 165, 300, 317, 318, 322, 45, 323, 324, 63, 303, 167, 9, 325, 315, 326, 327, 21], [61, 298, 180, 328, 310, 300, 7, 329, 9, 330, 192, 331, 332, 167, 303, 68, 184, 333, 21], [61, 297, 298, 334, 303, 68, 335, 1, 336, 11, 337, 338, 339, 13, 296, 297, 308, 93, 340, 341, 342, 343, 344, 11, 303, 21], [345, 125, 126, 346, 3, 24, 45, 347, 1, 9, 348, 349, 350, 167, 351, 345, 352, 353, 354, 355, 21], [198, 126, 356, 357, 59, 358, 359, 133, 346, 360, 150, 361, 362, 9, 363, 364, 1, 365, 21]]
val_labels: [[0, 0, 4,

In [ ]:
def data_iterator(sentences, labels, total_size: int, batch_size: int, shuffle: bool=False):
    # make a list that decides the order in which we go over the data- this avoids explicit shuffling of data
    order = list(range(total_size))
    if shuffle:
        random.seed(230)
        random.shuffle(order)

    # one pass over data
    for i in range((total_size+1)//batch_size):
        # fetch sentences and tags
        batch_sentences = [sentences[idx] for idx in order[i*batch_size:(i+1)*batch_size]]
        batch_tags = [labels[idx] for idx in order[i*batch_size:(i+1)*batch_size]]

        # compute length of longest sentence in batch
        batch_max_len = max([len(s) for s in batch_sentences])

        # prepare a numpy array with the data, initialising the data with pad_ind and all labels with -1
        # initialising labels to -1 differentiates tokens with tags from PADding tokens
        batch_data = vocab['<pad>']*np.ones((len(batch_sentences), batch_max_len))
        batch_labels = -1*np.ones((len(batch_sentences), batch_max_len))

        # copy the data to the numpy array
        for j in range(len(batch_sentences)):
            cur_len = len(batch_sentences[j])
            batch_data[j][:cur_len] = batch_sentences[j]
            batch_labels[j][:cur_len] = batch_tags[j]

        # since all data are indices, we convert them to torch LongTensors
        batch_data, batch_labels = torch.LongTensor(batch_data), torch.LongTensor(batch_labels)

        # shift tensors to GPU if available
        # if params.cuda:
        #     batch_data, batch_labels = batch_data.cuda(), batch_labels.cuda()

        # convert them to Variables to record operations in the computational graph
        batch_data, batch_labels = Variable(batch_data), Variable(batch_labels)

        yield batch_data, batch_labels, batch_sentences

In [ ]:
class Net(nn.Module):
    """
    This is the standard way to define your own network in PyTorch. You typically choose the components
    (e.g. LSTMs, linear layers etc.) of your network in the __init__ function. You then apply these layers
    on the input step-by-step in the forward function. You can use torch.nn.functional to apply functions
    such as F.relu, F.sigmoid, F.softmax. Be careful to ensure your dimensions are correct after each step.

    You are encouraged to have a look at the network in pytorch/vision/model/net.py to get a better sense of how
    you can go about defining your own network.

    The documentation for all the various components available to you is here: http://pytorch.org/docs/master/nn.html
    """

    def __init__(self, vocab_size, embedding_dim, lstm_hidden_dim, number_of_tags):
        """
        We define an recurrent network that predicts the NER tags for each token in the sentence. The components
        required are:

        - an embedding layer: this layer maps each index in range(params.vocab_size) to a params.embedding_dim vector
        - lstm: applying the LSTM on the sequential input returns an output for each token in the sentence
        - fc: a fully connected layer that converts the LSTM output for each token to a distribution over NER tags

        Args:
            params: (Params) contains vocab_size, embedding_dim, lstm_hidden_dim
        """
        super(Net, self).__init__()

        # the embedding takes as input the vocab_size and the embedding_dim
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # the LSTM takes as input the size of its input (embedding_dim), its hidden size
        # for more details on how to use it, check out the documentation
        self.lstm = nn.LSTM(embedding_dim,
                            lstm_hidden_dim, batch_first=True)

        # the fully connected layer transforms the output to give the final output layer
        self.fc = nn.Linear(lstm_hidden_dim, number_of_tags)

    def forward(self, s):
        """
        This function defines how we use the components of our network to operate on an input batch.

        Args:
            s: (Variable) contains a batch of sentences, of dimension batch_size x seq_len, where seq_len is
               the length of the longest sentence in the batch. For sentences shorter than seq_len, the remaining
               tokens are PADding tokens. Each row is a sentence with each element corresponding to the index of
               the token in the vocab.

        Returns:
            out: (Variable) dimension batch_size*seq_len x num_tags with the log probabilities of tokens for each token
                 of each sentence.

        Note: the dimensions after each step are provided
        """
        #                                -> batch_size x seq_len
        # apply the embedding layer that maps each token to its embedding
        # dim: batch_size x seq_len x embedding_dim
        s = self.embedding(s)

        # run the LSTM along the sentences of length seq_len
        # dim: batch_size x seq_len x lstm_hidden_dim
        s, _ = self.lstm(s)

        # make the Variable contiguous in memory (a PyTorch artefact)
        s = s.contiguous()

        # reshape the Variable so that each row contains one token
        # dim: batch_size*seq_len x lstm_hidden_dim
        s = s.view(-1, s.shape[2])

        # apply the fully connected layer and obtain the output (before softmax) for each token
        s = self.fc(s)                   # dim: batch_size*seq_len x num_tags

        # apply log softmax on each token's output (this is recommended over applying softmax
        # since it is numerically more stable)
        return F.log_softmax(s, dim=1)   # dim: batch_size*seq_len x num_tags

In [ ]:
def loss_fn(outputs, labels):
    """
    Compute the cross entropy loss given outputs from the model and labels for all tokens. Exclude loss terms
    for PADding tokens.

    Args:
        outputs: (Variable) dimension batch_size*seq_len x num_tags - log softmax output of the model
        labels: (Variable) dimension batch_size x seq_len where each element is either a label in [0, 1, ... num_tag-1],
                or -1 in case it is a PADding token.

    Returns:
        loss: (Variable) cross entropy loss for all tokens in the batch

    Note: you may use a standard loss function from http://pytorch.org/docs/master/nn.html#loss-functions. This example
          demonstrates how you can easily define a custom loss function.
    """

    # reshape labels to give a flat vector of length batch_size*seq_len
    labels = labels.view(-1)

    # since PADding tokens have label -1, we can generate a mask to exclude the loss from those terms
    mask = (labels >= 0).float()

    # indexing with negative values is not supported. Since PADded tokens have label -1, we convert them to a positive
    # number. This does not affect training, since we ignore the PADded tokens with the mask.
    labels = labels % outputs.shape[1]

    num_tokens = int(torch.sum(mask))

    # compute cross entropy loss for all tokens (except PADding tokens), by multiplying with mask.
    return -torch.sum(outputs[range(outputs.shape[0]), labels]*mask)/num_tokens

In [ ]:
class RunningAverage:
    """A simple class that maintains the running average of a quantity

    Example:
    ```
    loss_avg = RunningAverage()
    loss_avg.update(2)
    loss_avg.update(4)
    loss_avg() = 3
    ```
    """

    def __init__(self):
        self.steps = 0
        self.total = 0

    def update(self, val):
        self.total += val
        self.steps += 1

    def __call__(self):
        return self.total / float(self.steps)

In [ ]:
def train(model, optimizer, loss_fn, data_iterator, metrics, num_steps):
    """Train the model on `num_steps` batches

    Args:
        model: (torch.nn.Module) the neural network
        optimizer: (torch.optim) optimizer for parameters of model
        loss_fn: a function that takes batch_output and batch_labels and computes the loss for the batch
        data_iterator: (generator) a generator that generates batches of data and labels
        metrics: (dict) a dictionary of functions that compute a metric using the output and labels of each batch
        params: (Params) hyperparameters
        num_steps: (int) number of batches to train on, each of size params.batch_size
    """

    # set model to training mode
    model.train()

    # summary for current training loop and a running average object for loss
    summ = []
    loss_avg = RunningAverage()

    # Use tqdm for progress bar
    t = trange(num_steps)
    for i in t:
        # fetch the next training batch
        train_batch, labels_batch, _ = next(data_iterator)

        # compute model output and loss
        output_batch = model(train_batch)
        loss = loss_fn(output_batch, labels_batch)

        # clear previous gradients, compute gradients of all variables wrt loss
        optimizer.zero_grad()
        loss.backward()

        # performs updates using calculated gradients
        optimizer.step()

        # Evaluate summaries only once in a while
        if i % 10 == 0:
            # extract data from torch Variable, move to cpu, convert to numpy arrays
            output_batch = output_batch.data.cpu().numpy()
            labels_batch = labels_batch.data.cpu().numpy()

            # compute all metrics on this batch
            summary_batch = {metric: metrics[metric](output_batch, labels_batch)
                             for metric in metrics}
            summary_batch['loss'] = loss.item()
            summ.append(summary_batch)

        # update the average loss
        loss_avg.update(loss.item())
        t.set_postfix(loss='{:05.3f}'.format(loss_avg()))

    # compute mean of all metrics in summary
    metrics_mean = {metric: np.mean([x[metric]
                                     for x in summ]) for metric in summ[0]}
    metrics_string = " ; ".join("{}: {:05.3f}".format(k, v)
                                for k, v in metrics_mean.items())
    print("- Train metrics: " + metrics_string)

In [ ]:
def evaluate(model, loss_fn, data_iterator, metrics, num_steps):
    """Evaluate the model on `num_steps` batches.

    Args:
        model: (torch.nn.Module) the neural network
        loss_fn: a function that takes batch_output and batch_labels and computes the loss for the batch
        data_iterator: (generator) a generator that generates batches of data and labels
        metrics: (dict) a dictionary of functions that compute a metric using the output and labels of each batch
        params: (Params) hyperparameters
        num_steps: (int) number of batches to train on, each of size params.batch_size
    """

    # set model to evaluation mode
    model.eval()

    # summary for current eval loop
    summ = []

    # compute metrics over the dataset
    for _ in range(num_steps):
        # fetch the next evaluation batch
        data_batch, labels_batch, _ = next(data_iterator)

        # compute model output
        output_batch = model(data_batch)
        loss = loss_fn(output_batch, labels_batch)

        # extract data from torch Variable, move to cpu, convert to numpy arrays
        output_batch = output_batch.data.cpu().numpy()
        labels_batch = labels_batch.data.cpu().numpy()

        # compute all metrics on this batch
        summary_batch = {metric: metrics[metric](output_batch, labels_batch)
                         for metric in metrics}
        summary_batch['loss'] = loss.item()
        summ.append(summary_batch)

    # compute mean of all metrics in summary
    metrics_mean = {metric:np.mean([x[metric] for x in summ]) for metric in summ[0]}
    metrics_string = " ; ".join("{}: {:05.3f}".format(k, v) for k, v in metrics_mean.items())
    print("- Eval metrics : " + metrics_string)
    return metrics_mean

In [ ]:
def train_and_evaluate(
        model,
        train_sentences,
        train_labels,
        val_sentences,
        val_labels,
        num_epochs: int,
        batch_size: int,
        optimizer,
        loss_fn,
        metrics
):
    for epoch in range(num_epochs):
        # Run one epoch
        print("Epoch {}/{}".format(epoch + 1, num_epochs))

        # compute number of batches in one epoch (one full pass over the training set)
        num_steps = (len(train_sentences) + 1) // batch_size
        train_data_iterator = data_iterator(
            train_sentences, train_labels, len(train_sentences), batch_size, shuffle=True)
        train(model, optimizer, loss_fn, train_data_iterator,
              metrics, num_steps)

        # Evaluate for one epoch on validation set
        num_steps = (len(val_sentences) + 1) // batch_size
        val_data_iterator = data_iterator(
            val_sentences, val_labels, len(val_sentences), batch_size, shuffle=False)
        val_metrics = evaluate(
            model, loss_fn, val_data_iterator, metrics, num_steps)

In [ ]:
def accuracy(outputs, labels):
    """
    Compute the accuracy, given the outputs and labels for all tokens. Exclude PADding terms.

    Args:
        outputs: (np.ndarray) dimension batch_size*seq_len x num_tags - log softmax output of the model
        labels: (np.ndarray) dimension batch_size x seq_len where each element is either a label in
                [0, 1, ... num_tag-1], or -1 in case it is a PADding token.

    Returns: (float) accuracy in [0,1]
    """

    # reshape labels to give a flat vector of length batch_size*seq_len
    labels = labels.ravel()

    # since PADding tokens have label -1, we can generate a mask to exclude the loss from those terms
    mask = (labels >= 0)

    # np.argmax gives us the class predicted for each token by the model
    outputs = np.argmax(outputs, axis=1)

    # compare outputs with labels and divide by number of tokens (excluding PADding tokens)
    return np.sum(outputs == labels)/float(np.sum(mask))


metrics = {
    'accuracy': accuracy,
    # could add more metrics such as accuracy for each token type
}

In [ ]:
# manually change vocab size (unique no. of words) and change label size (unique no. of labels) for now
model = Net(35180, 50, 50, 6)
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_and_evaluate(model, train_sentences, train_labels, val_sentences, val_labels, 100, 5, optimizer, loss_fn, metrics)

if (os.path.isfile("model_weights.pth")):
    model.load_state_dict(torch.load('model_weights.pth'))
else:
    torch.save(model.state_dict(), 'model_weights.pth')


Epoch 1/100


100%|██████████| 2/2 [00:00<00:00, 60.60it/s, loss=1.836]


- Train metrics: accuracy: 0.011 ; loss: 1.859
- Eval metrics : accuracy: 0.069 ; loss: 1.807
Epoch 2/100


100%|██████████| 2/2 [00:00<00:00, 108.67it/s, loss=1.801]


- Train metrics: accuracy: 0.053 ; loss: 1.817
- Eval metrics : accuracy: 0.168 ; loss: 1.773
Epoch 3/100


...

- Train metrics: accuracy: 0.989 ; loss: 0.058
- Eval metrics : accuracy: 0.990 ; loss: 0.077
Epoch 99/100


100%|██████████| 2/2 [00:00<00:00, 117.65it/s, loss=0.075]


- Train metrics: accuracy: 0.989 ; loss: 0.056
- Eval metrics : accuracy: 0.990 ; loss: 0.075
Epoch 100/100


100%|██████████| 2/2 [00:00<00:00, 111.11it/s, loss=0.073]

- Train metrics: accuracy: 0.989 ; loss: 0.055
- Eval metrics : accuracy: 0.995 ; loss: 0.073


In [ ]:
# test_sentences_file = "stanford_cs230_medium_data/test/sentences.txt"
# test_labels_file = "stanford_cs230_medium_data/test/labels.txt"
test_sentences_file = "stanford_cs230_small_data/test/sentences.txt"
test_labels_file = "stanford_cs230_small_data/test/labels.txt"

sentences_w_words = []
labels = []

test_sentences = []
test_labels = []

with open(test_sentences_file) as f:
    for sentence_words in f.read().splitlines():
        #replace each token by its index if it is in vocab
        #else use index of UNK
        sentences_w_words.append(sentence_words)
        s = [vocab[token] if token in vocab
             else vocab['UNK']
             for token in sentence_words.split(' ')]
        test_sentences.append(s)

with open(test_labels_file) as f:
    for sentence_labels in f.read().splitlines():
        #replace each label by its index
        l = [tag_map[label] for label in sentence_labels.split(' ')]
        test_labels.append(l)
        labels.append(sentence_labels)

print(f"sentences_w_words: {sentences_w_words}")
print(f"labels: {labels}")
print(f"test_sentences: {test_sentences}")
print(f"test_labels: {test_labels}")

sentences_w_words: ['At the Group of Eight summit in Scotland , Japanese Prime Minister Junichiro Koizumi said he is outraged by the London attacks .', 'He noted terrorist acts must not be forgivable .', 'Sarin gas attacks on the Tokyo subway system in 1995 killed 12 people and injured thousands .', 'A human rights group has called on Asian leaders to increase pressure on Burma to hasten democratic reforms and stop human rights abuses .', 'The Alternative ASEAN Network for Burma said officials from the Association of Southeast Asian Nations meeting this week should consider new options in dealing with Burma .', 'It said leaders should consider supporting a possible resolution on Burma by the United Nations Security Council .', "The group also urged ASEAN leaders to acknowledge the many security problems caused by Burma 's military regime .", "The rights group accuses Burma 's government of involvement in illegal drug trafficking and human rights abuses , especially against some ethnic 

In [ ]:
print(len(test_sentences))
test_data_iterator = data_iterator(test_sentences, test_labels, len(test_sentences), 1, shuffle=True)
test_batch, labels_batch, abc = next(test_data_iterator)
model_output = model(test_batch)
print(model_output)
print(model_output.size())

10
tensor([[-2.2624e+00, -2.6768e+00, -7.1126e-01, -2.1636e+00, -2.3739e+00,
         -2.0552e+00],
        [-1.1266e-01, -3.9718e+00, -3.7686e+00, -4.1318e+00, -3.8317e+00,
         -3.6158e+00],
        [-1.1406e-02, -6.1174e+00, -6.0855e+00, -6.7167e+00, -6.6568e+00,
         -5.4341e+00],
        [-1.7969e-02, -5.2489e+00, -5.7480e+00, -6.5842e+00, -6.8024e+00,
         -4.9801e+00],
        [-1.5461e-03, -7.8946e+00, -8.5914e+00, -8.5861e+00, -7.6230e+00,
         -8.0766e+00],
        [-9.0880e-04, -7.8063e+00, -9.4739e+00, -9.6771e+00, -8.5972e+00,
         -8.6388e+00],
        [-4.7666e-03, -6.4018e+00, -7.2364e+00, -7.1027e+00, -8.5077e+00,
         -6.6062e+00],
        [-2.2682e-03, -6.9688e+00, -7.9802e+00, -8.0013e+00, -8.5516e+00,
         -7.6966e+00],
        [-9.3178e-04, -8.1316e+00, -9.0915e+00, -9.1219e+00, -8.4203e+00,
         -8.5421e+00],
        [-1.4258e-03, -7.6331e+00, -8.6984e+00, -8.7549e+00, -7.8302e+00,
         -8.4287e+00],
        [-1.7869e-03, -7.75

In [ ]:
print(f"model_output \n {model_output}")
# predicted_labels = torch.argmax(model_output, dim=1)
# print(f"predicted_labels \n {predicted_labels}")
# predicted_labels = torch.argmax(torch.abs(model_output), dim=1)
# print(f"predicted_labels \n {predicted_labels}")
predicted_labels = np.argmax(model_output.detach().numpy(), axis=1)
print(f"sentences_w_words \n {abc}")
print(f"predicted_labels \n {predicted_labels}")
# print(f"correct_labels \n {}")

model_output 
 tensor([[-2.2624e+00, -2.6768e+00, -7.1126e-01, -2.1636e+00, -2.3739e+00,
         -2.0552e+00],
        [-1.1266e-01, -3.9718e+00, -3.7686e+00, -4.1318e+00, -3.8317e+00,
         -3.6158e+00],
        [-1.1406e-02, -6.1174e+00, -6.0855e+00, -6.7167e+00, -6.6568e+00,
         -5.4341e+00],
        [-1.7969e-02, -5.2489e+00, -5.7480e+00, -6.5842e+00, -6.8024e+00,
         -4.9801e+00],
        [-1.5461e-03, -7.8946e+00, -8.5914e+00, -8.5861e+00, -7.6230e+00,
         -8.0766e+00],
        [-9.0880e-04, -7.8063e+00, -9.4739e+00, -9.6771e+00, -8.5972e+00,
         -8.6388e+00],
        [-4.7666e-03, -6.4018e+00, -7.2364e+00, -7.1027e+00, -8.5077e+00,
         -6.6062e+00],
        [-2.2682e-03, -6.9688e+00, -7.9802e+00, -8.0013e+00, -8.5516e+00,
         -7.6966e+00],
        [-9.3178e-04, -8.1316e+00, -9.0915e+00, -9.1219e+00, -8.4203e+00,
         -8.5421e+00],
        [-1.4258e-03, -7.6331e+00, -8.6984e+00, -8.7549e+00, -7.8302e+00,
         -8.4287e+00],
        [-1.786